In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import pandas as pd


# XGB Regressor to predict revenue for Steam games
# By Willyam Ramos

In [ ]:
# Reads matrix ready for regression and final merged data
X = pd.read_pickle("../data/processed/matrix_ready_for_regression.pkl")
data = pd.read_pickle("../data/processed/final_merged_data.pkl")

In [ ]:
# Remove F2P games (they have $0 revenue, not useful for prediction)
# $0 revenue would skew the regression results by creating a large cluster of points at zero and attributing profitability to features that indicate F2P status

y = data["log_estimated_revenue"]
mask_paid = X['f2p_flag'] == 0
X_paid = X[mask_paid].copy()
y_paid = y[mask_paid].copy()

In [ ]:
# Target variable in log scale
X_paid.columns

In [ ]:
# Drop f2p_flag and related features
cols_to_drop = ['f2p_flag', 'peak_players', 'copies_sold_reviews_proxy', 'user_reviews']
if 'Genres_Free To Play' in X_paid.columns:
    cols_to_drop.append('Genres_Free To Play')

X_paid = X_paid.drop(columns=cols_to_drop)

print(f"Original dataset: {len(X)} games")
print(f"Paid games only: {len(X_paid)} games")
print(f"Removed features: {cols_to_drop}")

In [ ]:
# Split data into training and testing sets (80% train, 20% test) 
X_train, X_test, y_train, y_test = train_test_split(
    X_paid, y_paid, test_size=0.2, random_state=42
)

In [ ]:
# Creates and trains the XGBoost regression model
model = XGBRegressor(
    n_estimators=500, # Number of trees
    learning_rate=0.05, # Step size shrinkage, or how much each tree is allowed to correct the previous one
    max_depth=8, # Maximum tree depth for base learners, not overall tree depth
    subsample=0.8, # Trains each tree on 80% of the data to prevent overfitting
    colsample_bytree=0.8, # Use 80% of the features for each tree
    objective="reg:squarederror", # Regression with squared loss
    tree_method="hist", # Histogram based algorithms are known to be faster for large datasets
    random_state=42, 
    early_stopping_rounds=50,  # Stop training if no improvement in 50 rounds
    eval_metric="rmse" # Evaluation metric is Root Mean Squared Error (RMSE)
)

# X_train and y_train correspond to the dataset attributes and target variable
model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

# Process: Build tree to predict y_train, then check performance on X_test and y_test, build another tree if not good enough, repeat until early stopping or max trees reached
# Each tree can be thought of as a weak learner that corrects the errors of the previous trees and has a max depth of 8 levels

In [ ]:
# Model predicts LOG revenue, and we convert it back to actual revenue
y_pred_log = model.predict(X_test)
y_pred_revenue = np.exp(y_pred_log) 

# Also convert actual values back to dollars for comparison
y_test_revenue = np.exp(y_test)

In [ ]:
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log)) # RMSE on log scale 
mae_log = mean_absolute_error(y_test, y_pred_log) # MAE on log scale
r2_log = r2_score(y_test, y_pred_log) # R² on log scale

# Metrics on ACTUAL dollar scale (for interpretability), will be larger due to exponentiation
rmse_actual = np.sqrt(mean_squared_error(y_test_revenue, y_pred_revenue))
mae_actual = mean_absolute_error(y_test_revenue, y_pred_revenue)
r2_actual = r2_score(y_test_revenue, y_pred_revenue)

print("=" * 50)
print("REGRESSION PERFORMANCE (Revenue Prediction)")
print("=" * 50)
print("\nLog-Scale Metrics (what model optimizes):")
print(f"  RMSE: {rmse_log:.4f}")
print(f"  MAE:  {mae_log:.4f}")
print(f"  R²:   {r2_log:.4f}")

print("\nActual Dollar Metrics (for interpretation):")
print(f"  RMSE: ${rmse_actual:,.2f}")
print(f"  MAE:  ${mae_actual:,.2f}")
print(f"  R²:   {r2_actual:.4f}")

In [ ]:

SUCCESS_THRESHOLD = 100_000  

LOG_SUCCESS_THRESHOLD = np.log1p(SUCCESS_THRESHOLD)

# Produces boolean arrays indicating whether each game is a "success" based on the threshold
y_test_success = (y_test >= LOG_SUCCESS_THRESHOLD).astype(int)
y_pred_success = (y_pred_log >= LOG_SUCCESS_THRESHOLD).astype(int)


In [ ]:
# Calculate accuracy (correct predictions / total predictions)
accuracy = accuracy_score(y_test_success, y_pred_success)

# Checks if we have both 0 and 1 in predictions and actuals
unique_pred = np.unique(y_pred_success)
unique_actual = np.unique(y_test_success)

print("\n" + "=" * 50)
print(f"CLASSIFICATION PERFORMANCE (Success >= ${SUCCESS_THRESHOLD:,})")
print(f"(Log threshold: {LOG_SUCCESS_THRESHOLD:.4f})")
print("=" * 50)

# Prints how many rows passed the success threshold over total rows
print(f"\nActual successes in test set: {y_test_success.sum()} / {len(y_test_success)} ({y_test_success.mean():.1%})")
print(f"Predicted successes: {y_pred_success.sum()} / {len(y_pred_success)} ({y_pred_success.mean():.1%})")

# Convert back to original scale for interpretability
y_test_revenue_display = np.expm1(y_test)
print(f"\nRevenue distribution in test set (original scale):")
print(f"  Min:     ${y_test_revenue_display.min():,.2f}")
print(f"  25th %:  ${np.percentile(y_test_revenue_display, 25):,.2f}")
print(f"  Median:  ${np.percentile(y_test_revenue_display, 50):,.2f}")
print(f"  75th %:  ${np.percentile(y_test_revenue_display, 75):,.2f}")
print(f"  Max:     ${y_test_revenue_display.max():,.2f}")
print(f"  Mean:    ${y_test_revenue_display.mean():,.2f}")

# Only calculate metrics if both classes exist
if len(unique_pred) > 1 and len(unique_actual) > 1:
    # Precision = TP / (All positives predicted)
    precision = precision_score(y_test_success, y_pred_success, zero_division=0)
    # Recall = TP / (All actual positives)
    recall = recall_score(y_test_success, y_pred_success, zero_division=0)
    # F1 Score = 2 * (Precision * Recall) / (Precision + Recall); Measures the balance between precision and recall
    f1 = f1_score(y_test_success, y_pred_success, zero_division=0)
    cm = confusion_matrix(y_test_success, y_pred_success)
    
    print(f"\nAccuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f} (of predicted successes, % actually successful)")
    print(f"Recall:    {recall:.4f} (of actual successes, % we caught)")
    print(f"F1 Score:  {f1:.4f}")
    
    print("\nConfusion Matrix:")
    print(f"                 Predicted")
    print(f"               Fail  Success")
    print(f"Actual Fail    {cm[0,0]:4d}    {cm[0,1]:4d}")
    print(f"      Success  {cm[1,0]:4d}    {cm[1,1]:4d}")
else:
    print(f"\nAccuracy:  {accuracy:.4f}")
    print("\n⚠️  WARNING: Model predicts only one class!")
    print(f"   Try adjusting SUCCESS_THRESHOLD (current: ${SUCCESS_THRESHOLD:,})")
    print(f"   Revenue range in test set: ${y_test_revenue_display.min():,.2f} to ${y_test_revenue_display.max():,.2f}")
    print(f"   Median revenue: ${np.median(y_test_revenue_display):,.2f}")
    print(f"   Mean revenue: ${y_test_revenue_display.mean():,.2f}")

In [ ]:
# This shows the top 20 most important features used by the model by importance score
# By default, the importance is calculated based on the number of times a feature is used to split the data across all trees (weight)
importances = pd.Series(model.feature_importances_, index=X_paid.columns)
top20 = importances.sort_values(ascending=False).head(20)

print("\n" + "=" * 50)
print("TOP 20 MOST IMPORTANT FEATURES")
print("=" * 50)
for feature, importance in top20.items():
    print(f"{feature:40s} {importance:.6f}")



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10, 8))
top20.sort_values().plot(kind='barh')  # sort bottom-to-top for best readability

plt.title("Top 20 Most Important Features (XGBoost)")
plt.xlabel("Feature Importance Score")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()